In [56]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
from futures import openWeatherKey
import time
from datetime import datetime
from meteostat import Point, Daily

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + openWeatherKey

In [68]:
# Load in CSV of cities, and their coordinates. Add in the part of the globe (North & South)
cdf = pd.read_csv('data/crop_cities.csv')
cdf['Hemisphere'] = np.where(cdf['lat']>=0,'North', 'South' )
weat = []
corn = []

for x in cdf.values:
    if x[3] == 'Wheat':
        weat.append(x[0])
    else:
        corn.append(x[0])

cdf.head()

,State,lat,lng,Crop,Hemisphere
0,Kansas,39.099789,-94.578560,Wheat,North
1,North Dakota,46.620146,-100.540737,Wheat,North
2,Montanna,43.427000,-71.587080,Wheat,North
3,Iowa,41.878000,-93.097702,Corn,North
4,illinois,39.739300,-89.266510,Corn,North


In [75]:
start = datetime(1980, 1, 1)
end = datetime.today()

d = {}
df = cdf.copy()
for x in df.values:
    # City Lat, Lng, Hemisphere
    x = list(x)
    dpoint = Point(x[1], x[2])
    out = Daily(dpoint, start, end).fetch()
    out['state'] = x[0]
    out['lat'] = x[1]
    out['lng'] = x[2]
    d[x[0]] = out

In [76]:
d.keys()

dict_keys(['Kansas', 'North Dakota', 'Montanna', 'Iowa', 'illinois', 'Nebraska', 'Minnesota'])

In [77]:
for x in list(d.keys()):
    print(len(d[x]['tavg']))

15528
15528
15528
6031
4088
6030
15528


In [126]:
d['North Dakota'].sort_values('prcp', ascending = False)

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,state,lat,lng
time,,,,,,,,,,,,,
1998-08-21,23.3,15.6,30.6,117.6,0.0,105.0,21.6,NaN,1020.1,563.0,North Dakota,46.620146,-100.540737
1999-08-12,13.3,10.6,15.6,113.8,0.0,NaN,25.9,NaN,1010.5,117.0,North Dakota,46.620146,-100.540737
1993-07-15,15.7,13.9,17.8,109.7,0.0,108.0,21.2,63.0,1016.0,0.0,North Dakota,46.620146,-100.540737
1994-09-15,16.9,12.8,19.4,109.5,0.0,294.0,15.1,50.0,1005.7,42.0,North Dakota,46.620146,-100.540737
2009-06-15,18.2,13.9,25.0,81.0,0.0,124.0,17.6,NaN,NaN,NaN,North Dakota,46.620146,-100.540737
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-07-20,25.0,17.8,31.7,0.0,0.0,NaN,11.5,NaN,1010.4,674.0,North Dakota,46.620146,-100.540737
1981-03-20,-0.5,-7.8,6.1,0.0,0.0,NaN,15.4,44.3,1010.3,294.0,North Dakota,46.620146,-100.540737
1981-03-19,0.4,-6.7,6.1,0.0,0.0,NaN,16.2,42.5,1020.8,708.0,North Dakota,46.620146,-100.540737


In [94]:
weat_weather = []
corn_weather = []

for x in list(d.keys()):
    if x in weat: 
        weat_weather.append(d[x])
    else:
        corn_weather.append(d[x])

In [95]:
weat_df = pd.concat(weat_weather).sort_index()
corn_df = pd.concat(corn_weather).sort_index()

In [96]:
weat_df = weat_df.groupby(weat_df.index).mean()
corn_df = corn_df.groupby(corn_df.index).mean()

In [97]:
weat_df.to_csv('data/Wheat_weather_daily.csv')
corn_df.to_csv('data/Corn_weather_daily.csv')


#for x in list(d.keys()):d[x].to_csv(f'data/CropCities/{x}.csv')

In [92]:
weat_weather[2]

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,state,lat,lng
time,,,,,,,,,,,,,
1980-01-01,-4.8,-12.2,2.8,0.0,0.0,NaN,3.7,20.5,1013.5,528.0,Montanna,43.427,-71.58708
1980-01-02,-3.9,-11.7,3.3,0.0,0.0,NaN,9.6,44.3,1012.2,342.0,Montanna,43.427,-71.58708
1980-01-03,-6.2,-11.1,-3.3,0.0,0.0,318.0,18.5,50.0,1018.5,546.0,Montanna,43.427,-71.58708
1980-01-04,NaN,-12.8,-1.7,0.0,0.0,NaN,NaN,33.5,NaN,528.0,Montanna,43.427,-71.58708
1980-01-05,-8.3,-15.6,-2.8,0.0,0.0,NaN,8.0,33.5,1016.2,264.0,Montanna,43.427,-71.58708
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-02,23.7,18.3,29.0,10.5,NaN,254.0,13.0,NaN,1011.0,NaN,Montanna,43.427,-71.58708
2022-07-03,21.0,15.0,27.2,0.0,NaN,310.0,8.5,NaN,1015.0,NaN,Montanna,43.427,-71.58708
2022-07-04,20.0,13.3,26.1,0.0,NaN,291.0,8.0,NaN,1019.2,NaN,Montanna,43.427,-71.58708


In [98]:
solar_cycle_wiki = 'https://en.wikipedia.org/wiki/List_of_solar_cycles'

In [102]:
solar_cycle_list = pd.read_html(solar_cycle_wiki)[0]

In [107]:
solar_cycle_list = solar_cycle_list[['Solar Cycle','Start (Minimum)', 'Maximum']].loc[0:24]

In [114]:
solar_cycle_list['Start (Minimum)'] = pd.to_datetime(solar_cycle_list['Start (Minimum)'])
solar_cycle_list['Maximum'] = pd.to_datetime(solar_cycle_list['Maximum'])

In [121]:
import datetime as dt 
solar_cycle_list['startYear'] = solar_cycle_list['Start (Minimum)'].dt.year
solar_cycle_list['startMonth'] = solar_cycle_list['Start (Minimum)'].dt.month
solar_cycle_list['maxYear'] = solar_cycle_list['Maximum'].dt.year
solar_cycle_list['maxMonth'] = solar_cycle_list['Maximum'].dt.month


In [124]:
solar_cycle_list[['Solar Cycle', 'startYear', 'startMonth', 'maxYear', 'maxMonth']].to_csv('data/Solar_cycle_list.csv')